In [344]:
import pandas as pd
import numpy as np
import plotly.express as px
import os
import matplotlib.pyplot as plt
from pandas.api.types import CategoricalDtype
from fuzzywuzzy import fuzz, process
from numpy import int8

# Define the 'category' data type
category = CategoricalDtype()


src = 'finalCCRSdata/'


In [326]:
def replace_test_names(df, replacements):
    # Loop over the keys and values of the replacements dictionary
    for old_value, new_value in replacements.items():
        # Create a boolean mask to select the rows where "TestName" contains the old value
        mask = df['TestName'].str.contains(old_value)

        # Update the "TestName" column for the selected rows to the new value
        df.loc[mask, 'TestName'] = new_value
    return df


def replace_units(df, replacements):
    for old_value, new_value in replacements.items():
        mask = df['Units'] == old_value
        df.loc[mask, 'Units'] = new_value
    return df


def replace_TestValue(df, replacements):
    for old_value, new_value in replacements.items():
        mask = df['TestValue'] == old_value
        df.loc[mask, 'TestValue'] = new_value
    return df


In [327]:
file_list = os.listdir(src)

# get a list of prefixes
# prefix_list = list(set([file.split('_')[0] for file in file_list if '_' in file and file.endswith('.csv')]))
prefix_list = ['LabResult']


# make a dictionary of dataframes
for prefix in prefix_list:
    prefix_files = [file for file in file_list if file.startswith(
        prefix) and file.endswith('.csv')]
    df_list = [pd.read_csv(os.path.join(src, file)) for file in prefix_files]
    df_concat_LabResult = pd.concat(
        df_list, keys=[(prefix, file) for file in prefix_files], ignore_index=False)


/tmp/ipykernel_45346/1505006405.py:11: DtypeWarning: Columns (7,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df_list = [pd.read_csv(os.path.join(src, file)) for file in prefix_files]
/tmp/ipykernel_45346/1505006405.py:11: DtypeWarning: Columns (7,8,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df_list = [pd.read_csv(os.path.join(src, file)) for file in prefix_files]


In [328]:
# drop useless columns
df_concat_LabResult = df_concat_LabResult[['LabResultId', 'LabLicenseeId', 'LicenseeId', 'LabTestStatus',
                                           'InventoryId', 'TestName', 'TestDate', 'TestValue',
                                           'ExternalIdentifier',
                                           # 'IsDeleted', 'CreatedBy', 'CreatedDate',
                                           # 'UpdatedBy', 'UpdatedDate'
                                           ]]


In [329]:
df_concat_LabResult = df_concat_LabResult.loc[df_concat_LabResult['TestName'] != 'Terpineol']


In [330]:


replacement_TestNames = {
    'Dichlorvos': 'Pesticide - DDVP (ppm)',
    'DDVP': 'Pesticide - DDVP (ppm)',
    'Permethrins': 'Pesticide - Permethrins (ppm)',
    'Spinosads': 'Pesticide - Spinosads (ppm)',
    'Spinosad': 'Pesticide - Spinosads (ppm)',
    'Residual Solvent - 2-Propanol \(IPA\) \(ug/g\)': 'Residual Solvent - 2-Propanol (ug/g)',
    'Clofentizine': 'Clofentizene',
    'Dichloro-Methane': 'Dichloromethane',
    'Heptanea': 'Heptane',
    'Heptanes': 'Heptane',
    'Kresoxin-Methyl': 'Kresoxim-Methyl',
    'Phosemet': 'Phosmet',
    'Propanes': 'Propane',
    'Pyrethrin I': 'Pyrethrins',
    'n-Hexane': 'Hexanes',
    'Total Aflatoxins': 'Aflatoxins',
    'Ochratoxin A': 'Ochratoxin',
    'Salmonella spp.': 'Microbiological - Salmonella (CFU/g)',
    'As': 'Heavy Metal - Arsenic (ppm)',
    'Cd': 'Heavy Metal - Cadmium (ppm)',
    'Hg': 'Heavy Metal - Mercury (ppm)',
    'Pb': 'Heavy Metal - Lead (ppm)',
    'Potency - D9THCA\(\)': 'Potency - D9THCA (%)',
    'Potency - D9THCA \(\)': 'Potency - D9THCA (%)',
    'Potency - D9THC \(\)': 'Potency - D9THC (%)',
    'Potency - D9THC\(\)': 'Potency - D9THC (%)',
    'Potency - CBDA\(\)': 'Potency - CBDA (%)',
    'Potency - CBDA \(\)': 'Potency - CBDA (%)',
    'Potency - CBD\(\)': 'Potency - CBD (%)',
    'Potency - CBD \(\)': 'Potency - CBD (%)',
    'Potency - Total THC \(\)': 'Potency - Total THC (%)',
    'Potency - Total CBD \(\)': 'Potency - Total CBD (%)',
    'd9-THC': 'Potency - D9THC (%)',
    'd9-THCA': 'Potency - D9THCA (%)',
    'Total d9-THC': 'Potency - Total THC (%)',
    'd9-CBD': 'Potency - CBD (%)',
    'd9-CBDA': 'Potency - CBDA (%)',
    'Total d9-CBD': 'Potency - Total CBD (%)',
    'Kresoxim-methyl': 'Pesticide - Kresoxim-Methyl (ppm)',
    'iso-Propanol': 'Residual Solvent - Isopropanol (ppm)',
    'Residual Solvent - 2-Propanol \(IPA\) \(ug/g\)': 'Residual Solvent - Isopropanol (ppm)',
    'Microbial- I502 panel \(3\)-Bile Tolerant gram neg': 'Microbiological - BTGN(CFU/g)',
    'Moisture & Water Activity-Water Activity \(Aw\)\(%\)': 'Moisture Analysis - Water Activity (aw)',
    'Microbial- I502 panel \(3\)-Salmonella': 'Microbiological - Salmonella (CFU/g)',
    'Clofentizene': 'Pesticide - Clofentezine (ppm)',
    'Microbial- I502 panel \(3\)-E.coli': 'Microbiological - STEC (CFU/g)'

}

df_concat_LabResult = replace_test_names(
    df_concat_LabResult, replacement_TestNames)


In [331]:

df_concat_LabResult['TestDate'] = pd.to_datetime(
    df_concat_LabResult['TestDate'])

# Convert the TestValue column to float, capturing any errors


bad_values = df_concat_LabResult.loc[pd.to_numeric(
    df_concat_LabResult['TestValue'], errors='coerce').isna()].copy()
bad_values['OriginalTestValue'] = bad_values['TestValue']


df_concat_LabResult['TestValue'] = pd.to_numeric(
    df_concat_LabResult['TestValue'], errors='coerce')
df_concat_LabResult = df_concat_LabResult.loc[~df_concat_LabResult['TestValue'].isna(
)]


In [332]:
# Determine if the values which are not numeric are ok to be removed from dataset

bad_values.value_counts('OriginalTestValue')

# yes


OriginalTestValue
Not Tested    7371
DET             13
trace            3
.                1
.0823.10         1
.17.11           1
.24.21           1
.43.             1
.63.             1
30.53.           1
9.2.97           1
dtype: int64

In [333]:
df_concat_LabResult = df_concat_LabResult.astype({
    'LabResultId': str,
    'LabLicenseeId': str,
    'LicenseeId': str,
    'LabTestStatus': category,
    'InventoryId': str,
    'TestName': str,
    'TestValue': float,
    'ExternalIdentifier': str
})


In [334]:


# Find all unique prefixes in TestName column
prefixes = df_concat_LabResult['TestName'].str.extract(
    r'^([^-]+)\s?-').dropna().iloc[:, 0].unique()

# Split DataFrame into two based on prefix presence in TestName column
df_with_prefix = df_concat_LabResult[df_concat_LabResult['TestName'].str.extract(
    r'^([^-]+)\s?-').iloc[:, 0].isin(prefixes)]
df_without_prefix = df_concat_LabResult[~df_concat_LabResult['TestName'].str.extract(
    r'^([^-]+)\s?-').iloc[:, 0].isin(prefixes)]


# clean df which has testnames with prefixes, the preferred format

In [335]:
# Define the regex pattern
pattern = r'^([^-]+)\s?-([^(]+)(?:\(([^)]+)\))?$'

# Extract the new columns from the TestName column
new_cols = df_with_prefix['TestName'].str.extract(pattern)


# Add the new columns to the df_with_prefix DataFrame
df_with_prefix = df_with_prefix.assign(
    TestGroup=new_cols[0], Analyte=new_cols[1], Units=new_cols[2])


replacement_units = {
    'Aw': 'aw',
    'PPM': 'ppm',
    'g/kg': 'mg/g',
    'ea.': 'ea',
    'CFU/G': 'CFU/g',
}

df_with_prefix = replace_units(df_with_prefix, replacement_units)


In [336]:
# Define a function to compute the fuzzy match score between two strings
# Get the list of canonical test names from the 'TestName' column of the df_with_prefix DataFrame
canonical_names = df_with_prefix['TestName'].unique()

# Define a function to compute the closest canonical name for a given test name

# Define a function to compute the closest canonical name and match score for a given test name


def closest_name(name):
    match = process.extractOne(name, canonical_names)
    return match[0], match[1]


# Apply the closest_name function to the 'TestName' column of the df_without_prefix DataFrame to standardize the test names and print the match score
df_without_prefix[['NewTestName', 'MatchScore']
                  ] = df_without_prefix['TestName'].apply(closest_name).apply(pd.Series)


/tmp/ipykernel_45346/289796853.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_without_prefix[['NewTestName', 'MatchScore']
/tmp/ipykernel_45346/289796853.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_without_prefix[['NewTestName', 'MatchScore']


# Checking fuzzy match accuracy and applying corrections
the fuzzy match results were visually analyzed, and corrections that are noted are below.
the resulting corrections were implemented on `df_LabResults_concat` during the `replace_test_names` application

 - spinosads is incorrect, amended by removing the double dictionary entry in the replace_test_names input
 - Clofentezine is matched correctly at score 75
 - Dichlorvos isn't matched because the canonical is the abbreviation DDVP, so I added a dict input for replace_test_names
 - Total Abamectin is correctly matched score 86
 - Total Pyrethrins ''
 - Total MGK 264 ''
 - Terpineol is not matched correctly, but does it matter? score 50. going to consider removing since it is not a required test. not in canonical name list
 - looking at the values, I am going to assume that this is mg/g it is, however it's not useful to me so i dropped it early on.
 - Total Xylenes is matched correctly, score 86
 - Ethyl_Acetate is matched correctly, score 83
 - Total Pentanes ''
 - Total Hexanes ''
 - Total Butanes ''
 - E Coli is not matched correct, sent to 'Moisture Analysis - Moisture Content (%)', match 60
 - ['Heavy Metal - Arsenic (ppm)', 'Heavy Metal - Arsenic (ug/g)']
 - ['Heavy Metal - Cadmium (ppm)', 'Heavy Metal - Cadmium (ug/g)']
 - ['Heavy Metal - Mercury (ppm)', 'Heavy Metal - Mercury (ug/g)']
 - ['Heavy Metal - Lead (ppm)', 'Heavy Metal - Lead (ug/g)']
 - Salmonella spp. was incorrect, so I added it to the replace_test_names dict
 - what to do with the heavy metals, i don't know which unit to assign them do since I don't have the product type available atm. I'll make them all ppm since that's the generic units

# Matching `df_without_prefix` with `df_with_prefix`

In [337]:
df_without_prefix['TestName'] = df_without_prefix['NewTestName']
df_without_prefix = df_without_prefix.drop(
    columns=['NewTestName', 'MatchScore'])

# Define the regex pattern
pattern = r'^([^-]+)\s?-([^(]+)(?:\(([^)]+)\))?$'

# Extract the new columns from the TestName column
new_cols = df_without_prefix['TestName'].str.extract(pattern)


# Add the new columns to the df_without_prefix DataFrame
df_without_prefix = df_without_prefix.assign(
    TestGroup=new_cols[0], Analyte=new_cols[1], Units=new_cols[2])


replacement_units = {
    'Aw': 'aw',
    'PPM': 'ppm',
    'g/kg': 'mg/g',
    'ea.': 'ea',
    'CFU/G': 'CFU/g',
}

df_without_prefix = replace_units(df_without_prefix, replacement_units)


/tmp/ipykernel_45346/1737276122.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_without_prefix['TestName'] = df_without_prefix['NewTestName']


In [339]:
df_lab_results = df_with_prefix.append(df_without_prefix)


/tmp/ipykernel_45346/2096448669.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_lab_results = df_with_prefix.append(df_without_prefix)


In [345]:
boob = {
    'Pass': True,
    'Fail': False
}

df_lab_results['LabTestStatus'] = df_lab_results['LabTestStatus'].map(boob)


df_lab_results = df_lab_results.astype({
    'LabResultId': int,
    'LabLicenseeId': int8,
    'LicenseeId': int8,
    'LabTestStatus': bool,
    'InventoryId': int,
    'TestName': str,
    'TestValue': float,
    'ExternalIdentifier': str,
    'TestGroup': 'category',
    'Analyte': 'category',
    'Units': 'category'
})


In [346]:
df_lab_results.info()


<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1623838 entries, ('LabResult', 'LabResult_1.csv', 0) to ('LabResult', 'LabResult_0.csv', 999979)
Data columns (total 12 columns):
 #   Column              Non-Null Count    Dtype         
---  ------              --------------    -----         
 0   LabResultId         1623838 non-null  int64         
 1   LabLicenseeId       1623838 non-null  int8          
 2   LicenseeId          1623838 non-null  int8          
 3   LabTestStatus       1623838 non-null  bool          
 4   InventoryId         1623838 non-null  int64         
 5   TestName            1623838 non-null  object        
 6   TestDate            1623838 non-null  datetime64[ns]
 7   TestValue           1623838 non-null  float64       
 8   ExternalIdentifier  1623838 non-null  object        
 9   TestGroup           1623838 non-null  category      
 10  Analyte             1623838 non-null  category      
 11  Units               1623838 non-null  category      
dtypes: 

In [347]:
df_lab_results.to_parquet('lab_results.parquet', index=False)
